<a href="https://colab.research.google.com/github/lanyu0322/pedestrian_firstdraft_figures/blob/master/Workers_and_residents_500buffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import time
import numpy as np
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
# -- install geopandas
try:
  import geopandas as gp
except:
  !pip install geopandas

import pyproj

In [ ]:
# -- read in the weekday counts
camera_fname = os.path.join("drive", "My Drive", "lwir", "data", "nycdot", 
                            "cams_ft_wd.feather")
camera = pd.read_feather(camera_fname)
cam_len = len(camera)

In [ ]:
# # -- set the root directory for the project                                     
# rpath = os.path.join("drive", "My Drive", "lwir")

In [ ]:
# # -- important definition
# def read_lehd():

#     """ set the file names """
#     lpath = os.path.join(rpath, "data", "lehd")
#     wname = os.path.join(lpath, "ny_wac_S000_JT00_2017.csv") 
#     rname = os.path.join(lpath, "ny_rac_S000_JT00_2017.csv")

#     """ read in the data for NY state """
#     print("READ_LEHD: loading the worker and resident totals for NY state...")
#     wdata = pd.read_csv(wname)
#     rdata = pd.read_csv(rname)

#     """ rename the geocode for merging with the shape data """
#     wdata.rename(columns = {'w_geocode': 'GEOID10'}, inplace=True)
#     rdata.rename(columns = {'h_geocode': 'GEOID10'}, inplace=True)

#     """ merge the worker and residents """
#     cols = ["GEOID10", "C000"]
#     lehd = pd.merge(wdata[cols], rdata[cols], on="GEOID10", 
#                     suffixes=["_w", "_r"], how="outer").fillna(0)

#     """ set the total number of workers and residents """
#     lehd["total_p"] = lehd["C000_w"] + lehd["C000_r"]

#     """ load the shapes """
#     print("READ_LEHD: loading the census blocks for NY state...")
#     t0      = time.time()
#     cpath   = os.path.join("drive", "My Drive", "lwir", "data", "lehd", 
#                            "tl_2019_36_tabblock10")
#     cname   = os.path.join(cpath, "tl_2019_36_tabblock10.shp")
#     bl_full = gp.read_file(cname)
#     print("READ_LEHD:   elapsed time = {0}s" \
#               .format(round(time.time() - t0), 2))

#     """ convert the shape geo ID to integer """
#     bl_full["GEOID10"] = bl_full["GEOID10"].astype(int)

#     """ sub-select only New York City Counties """
#     print("READ_LEHD: sub-selecting only Manhattan...")
#     bl_mn = bl_full[bl_full["COUNTYFP10"] == "061"]

#     """ merge LEHD data and geographic data """
#     print("READ_LEHD: merging Manhattan census blocks with LODES...")
#     lehd_bl = bl_mn.merge(lehd, on="GEOID10")

#     return lehd_bl

# # -- define geometry covert to NYSP
# def convert_to_nyc(input_data):
#   input_data.geometry = input_data.geometry.to_crs(epsg=2263)

In [ ]:
# # -- read in the LEHD data                                                      
# try:
#   lehd
# except:
#   lehd = read_lehd()

READ_LEHD: loading the worker and resident totals for NY state...
READ_LEHD: loading the census blocks for NY state...
READ_LEHD:   elapsed time = 29s
READ_LEHD: sub-selecting only Manhattan...
READ_LEHD: merging Manhattan census blocks with LODES...


In [ ]:
def convert_to_nyc(input_data):
  input_data.geometry = input_data.geometry.to_crs(epsg=2263)

In [ ]:
# -- define helper function for integrating within a circle """
def integrate_geodata(geo, vals, lat, lon, rad):
  

  ll_nyspd = pyproj.Proj("epsg:2263", preserve_units=True)(lon, lat)


  circ = Point(ll_nyspd[0], ll_nyspd[1]).buffer(rad)
 

  inter = geo.intersection(circ)
  

  frac = inter.area / geo.area
  return (frac * vals).sum() 

In [ ]:
# -- read geoggraphic data
dpath   = os.path.join("drive", "My Drive", "lwir", "data", "lehd", 
                       "tl_2019_36_tabblock10/tl_2019_36_tabblock10.shp")
ct_full = gp.read_file(dpath)

# -- convert geographic ID to intege 
ct_full["GEOID10"] = ct_full["GEOID10"].astype(int)

# -- sub-select only New York City Counties 
cnums  = ["061"]
ind    = ct_full.COUNTYFP10.isin(cnums)
ct_nyc = ct_full[ind]

In [ ]:
# -- read in the working people and residents
""" working people and residents within 500ft """
wname = "drive/My Drive/lwir/data/lehd/ny_wac_S000_JT00_2017.csv"
rname = "drive/My Drive/lwir/data/lehd/ny_rac_S000_JT00_2017.csv"
wdata = pd.read_csv(wname)
rdata = pd.read_csv(rname)
wdata.rename(columns = {'w_geocode': 'GEOID10'}, inplace=True)
rdata.rename(columns = {'h_geocode': 'GEOID10'}, inplace=True)

# -- merge LEHD data
lehd = pd.merge(wdata[["GEOID10", "C000"]], rdata[["GEOID10", "C000"]], 
               on="GEOID10", suffixes=["_w", "_h"], how="outer").fillna(0)

# -- calculate total number of people 
lehd["total_p"] = lehd["C000_w"] + lehd["C000_h"]

In [ ]:
# -- merge LEHD data and geographic data 
lehd_ct = ct_nyc.merge(lehd, on="GEOID10")
print("init w+r epsg: ", lehd_ct.geometry.crs)
convert_to_nyc(lehd_ct)
print("converted w+r epsg: ", lehd_ct.geometry.crs)
wrtot_manhattan = np.zeros(cam_len)
working_manhattan = np.zeros(cam_len)
residents_manhattan = np.zeros(cam_len)

init w+r epsg:  epsg:4269
converted w+r epsg:  epsg:2263


In [ ]:
# -- calculating total number, working people and residents in manhattan
for i in range(cam_len):
  wrtot_manhattan[i] = integrate_geodata(lehd_ct.geometry, lehd_ct.total_p, 
                                         camera.lat[i], camera.lon[i], 500.)
 
  working_manhattan[i] = integrate_geodata(lehd_ct.geometry, lehd_ct.C000_w,
                                         camera.lat[i], camera.lon[i], 500.)
  residents_manhattan[i] = integrate_geodata(lehd_ct.geometry, lehd_ct.C000_h, 
                                         camera.lat[i], camera.lon[i], 500.)

In [ ]:
#create DataFrame holding working people and residents in manhattan
wr_manhattan = np.vstack([wrtot_manhattan, working_manhattan, 
                          residents_manhattan]).T
wr_manhattan = pd.DataFrame(wr_manhattan)
wr_manhattan.columns = ["wrtot_manhattan", "working_manhattan", 
                        "residents_manhattan"]

In [ ]:
# -- write to csv
oname = os.path.join("drive", "My Drive", "lwir", "data", "4_pop_fit", 
                     "wr_manhattan.csv")
wr_manhattan.to_csv(oname, index=False)